# Main

In [ ]:
import os.path
import pickle
import re
import importlib
# importlib.reload(visualization) # this reloads changes to external scripts

url_adress = "https://online.rwth-aachen.de/RWTHonline/ee/ui/ca2/app/desktop/#/slc.cm.reg/student/modules?$ctx=design=ca;lang=de&orgId=1"

filename = 'extracted_data.pickle'
# if os.path.isfile(filename):
#     # continue previous log file
#     with open(filename, 'rb') as handle:
#         # this contains all extracted data
#         curr_data_dict = pickle.load(handle)
# else:
#     print("File does not exist")

# TODO for now always create a new log file
# this contains all extracted data
curr_data_dict = {}

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# Initiate the browser
#browser = webdriver.Chrome(ChromeDriverManager().install())

chrome_options = Options()
#chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver.exe")

In [ ]:
from auction_lib.functions import get_auction_id, get_current_price, save_data_into_dict, \
                                  get_time_series, get_offer_metrics, get_auction_detail_id

def open_main_auction_site(url_adress):
    # Open the Website
    driver.get(url_adress)

    # # TODO Click to show 50 elements on the website
    # #driver.find_element_by_id('resultListLimit').click()
    # select = Select(driver.find_element_by_id('resultListLimit'))
    # # select by visible text
    # select.select_by_visible_text('50')
    # # select by value
    # select.select_by_value('50')

    # # Click to reload with 50 elss
    # driver.find_els_by_id('buttons').click()

Opens Chrome with the RWTHonline course list website (no account needed)

In [ ]:
from datetime import datetime
import time

now = datetime.now()
extraction_time = now.strftime("%Y-%m-%d_%H:%M:%S")
print("Current Time =", extraction_time)

# Part 1 ===========================================
# Open the Website
open_main_auction_site(url_adress)
time.sleep(0.1)

First test of extracting a text field from the website

In [ ]:
# Part 2 ===========================================
##Find elements with Selenium, by css
# use the chrome plugin Selector Gadget: https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb
element_css_selector = "ca-maintenance-banner-header-slot"
els = driver.find_element(By.CLASS_NAME, element_css_selector)

els

In [ ]:
print(els.text)
print(els.tag_name)
print(els.parent)
print(els.location)
print(els.size)

In [ ]:
driver.find_element(By.CLASS_NAME, "ca-maintenance-banner-header-slot").text

In [ ]:
# Part 3 ===========================================
# get the current semester of the course catalogue
class_name = "//*[contains(concat( \" \", @class, \" \" ), concat( \" \", \"ng-star-inserted\", \" \" ))]"
els = driver.find_element(By.XPATH, class_name)

splits = re.split("\\n", els.text)
print('Number of XPath hits', len(splits))
#print(splits)

current_semester = splits[6]
current_semester

In [ ]:
# Part 4 ===========================================
# get links to all courses of the current course catalogue page
# https://online.rwth-aachen.de/RWTHonline/ee/ui/ca2/app/desktop/#/slc.cm.reg/student/modules/detail/1105283/257
# https://online.rwth-aachen.de/RWTHonline/ee/ui/ca2/app/desktop/#/slc.cm.reg/student/modules/detail/1105274/257

elems = driver.find_elements_by_xpath("//a[@href]")
substring = "https://online.rwth-aachen.de/RWTHonline/ee/ui/ca2/app/desktop/#/slc.cm.reg/student/modules/detail"

link_list = []
for elem in elems:
    #print(elem)
    if str(substring) in str(elem.get_attribute("href")):
        #print(elem.get_attribute("href"))
        link_list.append(str(elem.get_attribute("href")))
    else:
        None

link_list = list(set(link_list))

print(len(link_list))

In [ ]:
link_list

# Continue here afterwards

ToDos:
- get links for each page
- open each course link
- extract data of each course
    - save it in a dict

In [ ]:
for i in range(0,len(els)):
    text = els[i].text

    # extracted information
    curr_id = get_auction_id(text)
    curr_price = get_current_price(text)

    #print(extraction_time, ":\t", curr_id, curr_price)
    curr_data_dict = save_data_into_dict(curr_data_dict, curr_id, extraction_time, curr_price)

# Part 3 ===========================================
# Get detailed information from each course

# iterate through all courses on the main page
links = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mainpageLink", " " ))]')
for link in links:
    if link.text == "Details":
        link_href = link.get_attribute('href')
        #print(link.text, link.get_attribute('href'))

        # open new tab
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])

        driver.get(link_href)

        # extract detailed metrics from course offer page
        element_css_selector = "#geb_uebersicht"
        el = driver.find_element_by_css_selector(element_css_selector)
        text = el.text

        highest_bidder, n_o_bids, n_o_clicks, n_o_watchers = get_offer_metrics(text)
        details_dict = {}
        details_dict["highest_bidder"] = highest_bidder
        details_dict["n_o_bids"] = n_o_bids
        details_dict["n_o_clicks"] = n_o_clicks
        details_dict["n_o_watchers"] = n_o_watchers

        curr_detailed_id = get_auction_detail_id(driver)
        curr_data_dict[curr_detailed_id][extraction_time]["details"] = details_dict

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    else:
        driver.switch_to.window(driver.window_handles[0])

# Part 4 ===========================================
# get current value of XYZ
XYZ_label = "XYZ"
if XYZ_label not in curr_data_dict: curr_data_dict[XYZ_label] = {}
curr_data_dict["XYZ"][extraction_time] = 0


# Part 5 ===========================================
# close crawler
if len(driver.window_handles)>0:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [ ]:
curr_data_dict

In [ ]:
# to save results, execute this cell
with open(filename, 'wb') as handle:
    pickle.dump(curr_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(filename, 'rb') as handle:
    b = pickle.load(handle)

print(curr_data_dict == b)